In [1]:
# http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('max_columns', 50)
%matplotlib inline


In [4]:
s = pd.Series([7, 'Heisenberg', 3.14, -1789710578, 'Happy Eating!'],
              index=['A', 'Z', 'C', 'Y', 'E'])

#The Series constructor can convert a dictonary as well, using the keys of the dictionary as its index.
d = {'Chicago': 1000, 'New York': 1300, 'Portland': 900, 'San Francisco': 1100,
     'Austin': 450, 'Boston': None}
cities = pd.Series(d)
#cities
# you can use the index to select specific items from the Series
cities['Chicago']
cities[['Chicago', 'Portland', 'San Francisco']]
#Or you can use boolean indexing for selection.
cities[cities < 1000] # actually, it use each item to iterate to compare with 1000, return this item when returning true.


Austin      450.0
Portland    900.0
dtype: float64

In [5]:
# what if you are not sure whether an item is in the Series? You can check using idiomatic Python
print ('Seattle' in cities)
cities / 3
np.square(cities)

False


Austin            202500.0
Boston                 NaN
Chicago          1000000.0
New York         1690000.0
Portland          810000.0
San Francisco    1210000.0
dtype: float64

In [6]:
# use own defined column names : cols
no_headers = pd.read_csv('peyton-passing-TDs-2012.csv', sep=',', header=None,
                         names=cols)

NameError: name 'cols' is not defined

### merge/join
Like SQL's JOIN clause, `pandas.merge` allows two DataFrames to be joined on one or more keys. The function provides a series of parameters (on, left_on, right_on, left_index, right_index) allowing you to specify the columns or indexes on which to join.

By default, `pandas.merge` operates as an inner join, which can be changed using the `how` parameter

how : {'left', 'right', 'outer', 'inner'}, default 'inner'

* left: use only keys from left frame (SQL: left outer join)
* right: use only keys from right frame (SQL: right outer join)
* outer: use union of keys from both frames (SQL: full outer join)
* inner: use intersection of keys from both frames (SQL: inner join)

In [8]:
left_frame = pd.DataFrame({'key': range(5), 
                           'left_value': ['a', 'b', 'c', 'd', 'e']})
right_frame = pd.DataFrame({'key': range(2, 7), 
                           'right_value': ['f', 'g', 'h', 'i', 'j']})
print(left_frame)
print('\n')
print(right_frame)

#
pd.merge(left_frame, right_frame, on='key', how='inner')
pd.merge(left_frame, right_frame, on='key', how='left')
pd.merge(left_frame, right_frame, on='key', how='right')
pd.merge(left_frame, right_frame, on='key', how='outer') # mysql does not support this.


   key left_value
0    0          a
1    1          b
2    2          c
3    3          d
4    4          e


   key right_value
0    2           f
1    3           g
2    4           h
3    5           i
4    6           j


,key,left_value,right_value
0,2,c,f
1,3,d,g
2,4,e,h


# Combining

pandas also provides a way to combine DataFrames along an axis - **pandas.concat**. While the function is equivalent to SQL's UNION clause, there's a lot more that can be done with it.

In [15]:
pd.concat([left_frame, right_frame])
pd.concat([left_frame, right_frame], axis=1)

,key,left_value,key,right_value
0,0,a,2,f
1,1,b,3,g
2,2,c,4,h
3,3,d,5,i
4,4,e,6,j


# Grouping


__converters: dict.optional__

+ Dict of functions for converting values in certain columns. Keys can either be integers or column labels

pandas **groupby** returns a DataFrameGroupBy object which has a variety of methods, many of which are similar to standard SQL aggregate functions.

Calling **count** returns the total number of NOT NULL values within each column. If we were interested in the total number of records in each group, we could use **size**.

In [47]:
headers = ['name', 'title', 'department', 'salary']
chicago = pd.read_csv(r'Current_Employee_Names__Salaries__and_Position_Titles.csv',
                     header=0,
                     names=headers,
                     converters={'salary': lambda x: float(x.replace('$', ''))})
# chicago.head()

by_dept = chicago.groupby('department')

print(by_dept.count().head()) # NOT NULL records within each column
print('\n')
print(by_dept.size().tail()) # total records for each department

                   name  title  salary
department                            
ADMIN HEARNG         39     39      39
ANIMAL CONTRL        67     67      67
AVIATION           1521   1521    1521
BOARD OF ELECTION   117    117     117
BOARD OF ETHICS       9      9       9


department
PUBLIC LIBRARY     961
STREETS & SAN     2152
TRANSPORTN        1073
TREASURER           22
WATER MGMNT       1840
dtype: int64


In [54]:
print(by_dept.sum()[20:22]) # total salaries of each department
print('\n')
print(by_dept.mean()[20:25]) # average salary of each department
print('\n')
print(by_dept.median()[20:25]) # take that, RDBMS!

                    salary
department                
HUMAN RESOURCES  5313360.0
INSPECTOR GEN    4680780.0


                         salary
department                     
ADMIN HEARNG       76829.846154
ANIMAL CONTRL      54977.498507
AVIATION           73619.429849
BOARD OF ELECTION  56175.487179
BOARD OF ETHICS    86697.333333


                    salary
department                
HUMAN RESOURCES    73170.0
INSPECTOR GEN      75036.0
IPRA               89880.0
LAW                71292.0
LICENSE APPL COMM  71292.0


In [56]:
# SELECT department, COUNT(DISTINCT title)
# FROM chicago
# GROUP BY department
# ORDER BY 2 DESC
# LIMIT 5;

by_dept.title.nunique().sort_values(ascending=False)[:5]

department
TRANSPORTN          162
WATER MGMNT         154
POLICE              122
AVIATION            122
GENERAL SERVICES    121
Name: title, dtype: int64

In [58]:
print len(chicago)

32062


In [62]:
print np.arange(len(chicago))+1

[    1     2     3 ..., 32060 32061 32062]


In [63]:
def ranker(df):
    """Assigns a rank to each employee based on salary, with 1 being the highest paid.
    Assumes the data is DESC sorted."""
    df['dept_rank'] = np.arange(len(df)) + 1
    return df


chicago.sort_values('salary', ascending=False, inplace=True)
chicago = chicago.groupby('department').apply(ranker)
print(chicago[chicago.dept_rank == 1].head(7))

                           name                                        title  \
8184           EVANS,  GINGER S                     COMMISSIONER OF AVIATION   
7954             EMANUEL,  RAHM                                        MAYOR   
25532         SANTIAGO,  JOSE A                            FIRE COMMISSIONER   
8042         ESCALANTE,  JOHN J                  FIRST DEPUTY SUPERINTENDENT   
19672          MORITA,  JULIA Y                       COMMISSIONER OF HEALTH   
23631           REIFMAN,  DAVID       COMMISSIONER OF PLANNING & DEVELOPMENT   
19721  MORRISON BUTLER,  LISA V  COMMISSIONER OF FAMILY AND SUPPORT SERVICES   

                  department    salary  dept_rank  
8184                AVIATION  300000.0          1  
7954          MAYOR'S OFFICE  216210.0          1  
25532                   FIRE  202728.0          1  
8042                  POLICE  197724.0          1  
19672                 HEALTH  177000.0          1  
23631  COMMUNITY DEVELOPMENT  175020.0         